# Building a database for crime reports

In this project, we will build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file `boston.csv`. The first four rows are shown here:

|incident_number| offense_code | description |date | day_of_the_week | lat | long |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1 | 619 |LARCENY ALL OTHERS | 2018-09-02 | Sunday | 42.35779134	| -71.13937053 |
| 2	| 1402 | VANDALISM | 2018-08-21 | Tuesday |	42.30682138	|-71.06030035 |
| 3|	3410|	TOWED MOTOR VEHICLE|	2018-09-03|	Monday|	42.34658879|	-71.07242943|
|4	|3114|	INVESTIGATE PROPERTY	|2018-09-03	|Monday	|42.33418175	|-71.07866441 |

The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

The goal of this project is to create a database named `crime_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the boston.csv file. We will be creating the table inside a schema named `crimes`. We will also create the `readonly` and `readwrite` groups with the appropriate privileges. Finally, we will also need to create one user for each of these groups.

The following diagram illustrates a high level overview of what we want to achieve:

![](server_db.png)

## Creating the Crime Database 

In [1]:
import psycopg2
conn_dq = psycopg2.connect(dbname="dq", user="dq")
cur_dq = conn_dq.cursor()

In [2]:
# Set autocommit to True as creation of database cannot be inside a transaction block
conn_dq.autocommit = True
cur_dq.execute("CREATE DATABASE crime_db")
conn_dq.autocommit = False

In [3]:
conn_dq.close()

In [4]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
conn.autocommit = True # to avoid transaction blocks

cur.execute("CREATE SCHEMA crimes")

## Obtaining the Column Names and Sample

We now have a database and a schema — we are ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

In [5]:
import csv
with open("boston.csv", "r") as ifile:
    reader = csv.reader(ifile)
    rows = list(reader)
    col_headers = rows[0]
    rows = rows[1:]
    first_row = rows[0]
    print(col_headers)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating an Auxiliary Function

Creating `get_col_value_set()` that returns a set of values for a given column index in a given csv file.

This function will be useful for two reasons:

- Checking whether an enumerated datatype might be a good choice for representing a column.
- Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

In [6]:
def get_col_value_set(csv_filename, index):
    with open(csv_filename, "r") as ifile:
        reader = csv.reader(ifile)
        rows = list(reader)
        rows = rows[1:]
        value_set = set()
        for row in rows:
            value_set.add(row[index])
        return value_set

In [7]:
value_sets = []
for index, col in enumerate(col_headers):
    value_set = get_col_value_set("boston.csv", index)
    print("Length of value set for col {} = {}".format(col, len(value_set)))

Length of value set for col incident_number = 298329
Length of value set for col offense_code = 219
Length of value set for col description = 239
Length of value set for col date = 1177
Length of value set for col day_of_the_week = 7
Length of value set for col lat = 18177
Length of value set for col long = 18177


## Analyzing the Maximum Length of the "description" column

In [8]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [9]:
desc_value_set = get_col_value_set("boston.csv", 2)
print(max([len(desc) for desc in desc_value_set]))

58


## Creating the Table

In [10]:
cur.execute("""
    CREATE TYPE enum_weekday AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday',
                                     'Thursday', 'Friday', 'Saturday')
""")

In [11]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [12]:
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week enum_weekday,
        lat DECIMAL,
        long DECIMAL
    )
""")

## Loading the Data

In [13]:
with open("boston.csv", "r") as ifile:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", ifile)

In [14]:
cur.execute("SELECT COUNT(*) FROM crimes.boston_crimes")
print(cur.fetchone())

(298329,)


## Revoking public privileges

Our goal is to create the two user groups: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the `public` group on the public schema and the database.

In [15]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public")

## Creating User Groups

We have made sure that we are not going to inadvertently inherit privileges from the public group. The next step is to create our two users groups.

### Creating group `readonly` and granting relevant permissions

In [16]:
cur.execute("CREATE GROUP readonly NOLOGIN")

In [17]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")

### Creating group `readwrite` and granting relevant permissions

In [18]:
cur.execute("CREATE GROUP readwrite NOLOGIN")

In [19]:
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite")

## Creating Users

In [20]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")

In [21]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")

## Testing

Test the database setup using SQL queries on the `pg_roles` table and `information_schema.table_privileges`.

In the `pg_roles` table we will check database related privileges and for that we will look at the following columns:

- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be False on every user / group that we have created.
- `rolcanlogin`: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In [22]:
# close the connection to test with a new connection
conn.close()

In [23]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
conn.autocommit = True # to avoid transaction blocks

In [24]:
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist')
""")

In [25]:
print(*cur.fetchall(), sep="\n")

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


In the `information_schema.table_privileges` we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [26]:
cur.execute("""
    SELECT grantee, privilege_type, table_name
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite')
""")

In [27]:
print(*cur.fetchall(), sep="\n")

('readonly', 'SELECT', 'boston_crimes')
('readwrite', 'INSERT', 'boston_crimes')
('readwrite', 'SELECT', 'boston_crimes')
('readwrite', 'UPDATE', 'boston_crimes')
('readwrite', 'DELETE', 'boston_crimes')
